This script is used to produce Figure S1 (trends and change in mean bloom start, end, and length between 1970 and 2100) for the Payne et al. paper "XXX". 

1. Import Packages

In [3]:
import numpy as np
import numpy.ma as ma
import xarray as xr
import glob
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import stats
import math
import matplotlib.path as mpath
import xesmf as xe

2. Read in a randomly chosen NPP file. This is used to generate latitude (lat) and longitude (lon) fields for CESM files.

In [4]:
# Select a file
vdir = 'ocn'
vnam = 'photoC_diat_zint_2'
fdir = '/glade/campaign/cgd/cesm/CESM2-LE/'+ vdir + '/proc/tseries/day_1/' + vnam + '/'
fnam = 'b.e21.BSSP370cmip6.f09_g17.LE2-1301.003.pop.h.ecosys.nday1.photoC_diat_zint_2.20450102-20550101.nc'

ncfile = xr.open_dataset(fdir + fnam)
lon = ncfile.TLONG.values
lat = ncfile.TLAT.values

# Interpolate lat and lon values over holes in the arrays
ok = ~np.isnan(lon)
xp = ok.ravel().nonzero()[0]
fp = lon[~np.isnan(lon)]
x  = np.isnan(lon).ravel().nonzero()[0]
lon[np.isnan(lon)] = np.interp(x, xp, fp)

ok = ~np.isnan(lat)
xp = ok.ravel().nonzero()[0]
fp = lat[~np.isnan(lat)]
x  = np.isnan(lat).ravel().nonzero()[0]
lat[np.isnan(lat)] = np.interp(x, xp, fp)


3. Read in bloom start (max_25) and end (max_25_2) files. These files were created for each year and across all ensemble members using the "BloomMetrics" script. Bloom start occurs on the date when biomass first surpasses 25% of the maximum biomass at any given grid cell, and end occurs on the date when biomass first diminishes below 25% of the maximum following the peak in biomass. Afterwards, I set all bloom start and end values to 'nan' if they are equal to zero.

In [5]:
max_25 = np.zeros([14,384,320,50]); max_25_2 = np.zeros([14,384,320,50]); 
for yrset in np.arange(0,14):
    yrnum = str(1970 + yrset*10)
    dir = np.loadtxt("/glade/u/home/cpayne/CESM2LE/AnnualGC/" + yrnum + "_max_25.txt")
    max_25[yrset,:,:,:] = dir.reshape(384,320,50)
    dir = np.loadtxt("/glade/u/home/cpayne/CESM2LE/AnnualGC/" + yrnum + "_max_25_2.txt")
    max_25_2[yrset,:,:,:] = dir.reshape(384,320,50)

max_25 = np.where(max_25==0, np.nan, max_25)
max_25_2 = np.where(max_25_2==0, np.nan, max_25_2)

5. Choose 

In [6]:
yr2 = 13
vari = max_25_2[0,:,:,:]-max_25[0,:,:,:]
vari2 = max_25_2[yr2,:,:,:]-max_25[yr2,:,:,:]

In [ ]:
valslp = np.full([384,320], np.nan); 
mnslp = np.full([384,320],np.nan);
import math
for i in np.arange(0,384):
    for j in np.arange(0,320):
        if np.isnan(max_25_2[0,i,j,0]) == False:
            ens_slp = np.zeros([50])
            lr = stats.linregress(np.arange(1970,2101,10),np.mean(max_25_2[:,i,j,:]-max_25[:,i,j,:],axis=1))
            #if lr.pvalue < 0.05:
            mnslp[i,j] = lr.slope
            for ens in np.arange(0,50):
                lr = stats.linregress(np.arange(1970,2101,10),max_25_2[:,i,j,ens]-max_25[:,i,j,ens])
                ens_slp[ens] = lr.slope
            ratio = abs(mnslp[i,j])/np.std(ens_slp)
            if ratio >= 2 /(math.sqrt(49)):
                valslp[i,j] = mnslp[i,j]


In [ ]:
# make circular boundary for polar stereographic circular plots
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

#fig = plt.figure(figsize=(3,3)) 
fig = plt.figure(figsize=(6,6)) 
ax = plt.axes(projection=ccrs.NorthPolarStereo()); 
ax.set_boundary(circle, transform=ax.transAxes)
ax.set_extent([-180, 180, 50, 90], crs = ccrs.PlateCarree());
pc = ax.pcolormesh(lon, lat, np.mean(vari2,axis=2)/130 - np.mean(vari,axis=2)/130, 
                   transform=ccrs.PlateCarree(), vmin = -1, vmax = 1, cmap='cmo.balance');
ax.coastlines();  ax.add_feature(cfeature.LAND); 
plt.colorbar(pc, orientation='horizontal', extend='both');
figdir = '/glade/u/home/cpayne/CESM2LE/Figures/FigS1cbar.png';
plt.savefig(figdir,facecolor='none', dpi = 600);
plt.close()

In [44]:
# make circular boundary for polar stereographic circular plots
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

fig = plt.figure(figsize=(3,3)) 
# fig = plt.figure(figsize=(6,6)) 
ax = plt.axes(projection=ccrs.NorthPolarStereo()); 
ax.set_boundary(circle, transform=ax.transAxes)
ax.set_extent([-180, 180, 50, 90], crs = ccrs.PlateCarree());
pc = ax.pcolormesh(lon, lat, valslp, 
                   transform=ccrs.PlateCarree(), vmin = -1, vmax = 1, cmap='cmo.balance');
ax.coastlines();  ax.add_feature(cfeature.LAND); #plt.colorbar(pc, orientation='horizontal', extend='both');
figdir = '/glade/u/home/cpayne/CESM2LE/Figures/FigS1f.png';
plt.savefig(figdir,facecolor='none', dpi = 600);
plt.close()